# Address Matching with Apache Spark

## Iniciar Spark

In [23]:
from pyspark.sql.session import SparkSession

spark = spark = SparkSession.builder \
    .appName("Adress Matching") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "20g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

In [24]:
conf = spark.sparkContext.getConf()
for (key, value) in conf.getAll():
  print(f"{key}: {value}")

spark.app.name: Adress Matching
spark.app.id: local-1710328292060
spark.app.startTime: 1710328289968
spark.executor.id: driver
spark.driver.host: 10.6.130.30
spark.driver.extraJavaOptions: -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
spark.

## Leer fichero

El dataset continene exactamente 24 columnas las cuales, el ***, almacena información relevante el trabajo que realizan. En cambio, en este proyecto, se utilizarán las siguientes columnas pues contienen la información más relevante:
- **uuid_idt**: contiene un identificador alfanumérico único para cada dirección.
- **tvia**: contiene el tipo de vía, es decir, si es una calle, avenida, carretera, etc.
- **nvia**: contiene el nombre de la vía.
- **numer**: contiene el número de la vía.
- **codmun**: está compuesto por un código numérico que identifica a un municipio, es decir, el código postal.
- **nommun**: contiene el nombre del municipio.
- **direccion**: contiene la dirección completa."

In [25]:
# Leer el archivo CSV y cargarlo en un DataFrame
file =  "../data/raw_data/TFM_Direcciones.tab"
first_df = spark.read.option("delimiter", "\t").option("encoding", "windows-1252").csv(file, header=True, inferSchema=True)

# Seleccionar solo las columnas deseadas
selected_columns = [
  "uuid_idt",
  "tvia",
  "nvia",
  "numer",
  "codmun",
  "nommun",
  "direccion",
]
first_df = first_df.select(selected_columns)

# Mostrar el DataFrame
first_df.show()
first_df.schema
size_df = first_df.count()

+--------------------+------------+--------------------+-----+------+------+--------------------+
|            uuid_idt|        tvia|                nvia|numer|codmun|nommun|           direccion|
+--------------------+------------+--------------------+-----+------+------+--------------------+
|027C0FF8-B17B-11E...|       CALLE|        FUENTE SANTA|    5| 35001|Agaete|CALLE FUENTE SANT...|
|028F2302-B17B-11E...|       CALLE|       CRUZ CHIQUITA|    2| 35001|Agaete|CALLE CRUZ CHIQUI...|
|02918516-B17B-11E...|URBANIZACION|RESIDENCIAL PALMERAL|    9| 35001|Agaete|URBANIZACION RESI...|
|03A306C0-7525-11E...|       CALLE|SEÑORITA MARIA MA...|    2| 35001|Agaete|CALLE SEÑORITA MA...|
|03A306C0-7525-11E...|       CALLE|SRTA M MANRIQUE LARA|    2| 35001|Agaete|CALLE SRTA M MANR...|
|03A306C0-7525-11E...|       CALLE| MARIA MANRIQUE LARA|    0| 35001|Agaete|CALLE MARIA MANRI...|
|03A306C0-7525-11E...|       CALLE| MARIA MANRIQUE LARA|    2| 35001|Agaete|CALLE MARIA MANRI...|
|03A306C0-7525-11E..

In [27]:
from pyspark.sql.functions import upper

file =  "../data/raw_data/data-09022024.csv"
second_df = spark.read.option('header', True).csv(file)

selected_columns = [
  "uuid_idt",
  "codmun",
  "nommun",
  "direccion"
]

second_df = second_df.select(selected_columns)
second_df = second_df.select(upper('uuid_idt').alias('uuid_idt'), 'codmun', upper('nommun').alias('nommun'), 'direccion')
second_df.show()
first_df.schema
size_df += second_df.count()

+--------------------+------+--------------------+--------------------+
|            uuid_idt|codmun|              nommun|           direccion|
+--------------------+------+--------------------+--------------------+
|c0cf3b94-0ad1-11e...| 35016|Las Palmas de Gra...|IGUAZU 42 0 LAS P...|
|73b57c1b-3251-11e...| 35016|Las Palmas de Gra...|PARQUE CENTRAL BL...|
|c70e3ec6-3eda-11e...| 35016|Las Palmas de Gra...|CONCEJAL GARCIA F...|
|687ba81e-3251-11e...| 35006|              Arucas|PEDRO MORALES DEN...|
|74921989-3251-11e...| 35019|San Bartolomé de ...|ISLA LOBOS 19 0 S...|
|6c0d19ff-3251-11e...| 35016|Las Palmas de Gra...|CALLE LEON Y CAST...|
|6360fdcd-3251-11e...| 35016|Las Palmas de Gra...|VIGEN PILAR 45 4 ...|
|758b3855-3251-11e...| 35016|Las Palmas de Gra...|CALLE PALMERA CAN...|
|5d36841a-7a6b-11e...| 35016|Las Palmas de Gra...|RUPERTO CHAPI 0 L...|
|6c00dbf4-3251-11e...| 35021|       Santa Brígida|MANUEL HERNANDEZ ...|
|733a49c5-3251-11e...| 35027|               Teror| HERRERIA 36 0

In [28]:
print(size_df)

2414514
